# ICP Alignment Accuracy Analysis against Ground Truth

## Objective
In this notebook, we perform **Point-to-Plane ICP** to align synthetic scans to a reference object. 
Crucially, we verify the accuracy by comparing the **Calculated Transformation** against the **Ground Truth Transformation** exported from Blender.

---

## The Math: Inverse vs. Direct Transformation

It is important to understand the direction of the matrices:

1.  **Ground Truth ($T_{gt}$):** This is the matrix that moved the object in Blender.
    $$ P_{source} = T_{gt} \times P_{reference} $$

2.  **ICP Result ($T_{icp}$):** ICP calculates how to move the Source **back** to the Reference.
    $$ P_{reference} \approx T_{icp} \times P_{source} $$

3.  **The Comparison:**
    To compare them, we must realize that $T_{icp}$ is essentially trying to be the **inverse** of $T_{gt}$.
    To check the accuracy, we calculate the **Estimated Pose** ($T_{est}$) by inverting the ICP result:
    $$ T_{est} = (T_{icp})^{-1} $$
    
    We then compare $T_{est}$ with $T_{gt}$.

## Error Metrics
* **Translation Error:** Euclidean distance between the estimated position and true position (in meters).
* **Rotation Error:** The angular difference between the estimated rotation and true rotation (in degrees).

In [ ]:
import open3d as o3d
import pandas as pd
import numpy as np
import os
import time
import math

print(f"Open3D version: {o3d.__version__}")

## 1. Configuration

In [ ]:
# Path to the data
DATA_ROOT = r"C:\Users\RobinSchool\Stichting Hogeschool Utrecht\MNLE Imagine Project - Documents\DataProgram\Blender_Generated_Data"
SCANS_DIR = os.path.join(DATA_ROOT, "scans")
INDEX_FILE = os.path.join(DATA_ROOT, "dataset_index.csv")

# Load Index
if os.path.exists(INDEX_FILE):
    df_index = pd.read_csv(INDEX_FILE)
    print(f"Dataset loaded: {len(df_index)} records.")
else:
    print("ERROR: Index file not found.")

## 2. Helper Functions (Math & Processing)
Here we define functions to calculate the rotation error in degrees.

In [ ]:
def get_rotation_error(R_est, R_gt):
    """
    Calculates the angular difference (geodesic distance) between two rotation matrices.
    Formula: theta = arccos( (trace(R_diff) - 1) / 2 )
    Returns error in Degrees.
    """
    # R_diff = R_est * R_gt^T
    R_diff = np.dot(R_est, R_gt.T)
    
    # Calculate Trace
    tr = np.trace(R_diff)
    
    # Clip to safety range [-1, 3] for numerical stability
    # (Trace of 3x3 rotation matrix is between -1 and 3)
    tr = np.clip(tr, -1, 3)
    
    rad = math.acos((tr - 1) / 2)
    return math.degrees(rad)

def get_translation_error(t_est, t_gt):
    """Euclidean distance between two vectors."""
    return np.linalg.norm(t_est - t_gt)

def parse_ground_truth_matrix(row):
    """Extracts the flattened m00-m33 columns and reshapes to 4x4."""
    # Filter columns starting with 'm' followed by digits
    cols = [f"m{r}{c}" for r in range(4) for c in range(4)]
    flat = row[cols].values.astype(float)
    matrix = flat.reshape(4, 4)
    
    # Normalize rotation (remove scale if present from Blender)
    # We divide the first 3 columns by their norm
    for col in range(3):
        norm = np.linalg.norm(matrix[:3, col])
        if norm > 0:
            matrix[:3, col] /= norm
            
    return matrix

def load_and_preprocess(filepath):
    pcd = o3d.io.read_point_cloud(filepath, format='xyz')
    # Estimate normals for Point-to-Plane ICP
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    return pcd

## 3. Load Reference (Sample 0)

In [ ]:
# The first sample is always our "Zero" / Reference state
ref_row = df_index.iloc[0]
ref_path = os.path.join(SCANS_DIR, ref_row['filename'])

pcd_target = load_and_preprocess(ref_path)
pcd_target.paint_uniform_color([0.5, 0.5, 0.5]) # Grey
print("Reference Loaded.")

## 4. Main Processing Loop
We loop through the dataset, run ICP, and compare the result with the Ground Truth.

In [ ]:
results = []

print("Starting Analysis...")

# Skip index 0 (Reference)
for idx, row in df_index.iloc[1:].iterrows():
    
    # A. Load Data
    filepath = os.path.join(SCANS_DIR, row['filename'])
    if not os.path.exists(filepath): continue
        
    pcd_source = load_and_preprocess(filepath)
    
    # B. Ground Truth (GT)
    # This is the matrix that moved the object in Blender.
    T_gt = parse_ground_truth_matrix(row)
    
    # C. Run ICP
    # We want to align Source -> Target
    # Ideally, the result T_icp should be the INVERSE of T_gt
    reg = o3d.pipelines.registration.registration_icp(
        pcd_source, pcd_target, max_correspondence_distance=0.5,
        init=np.eye(4),
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane()
    )
    
    T_icp = reg.transformation
    
    # D. Calculate Estimated Pose
    # If T_icp moves Source to Target, then the "Pose" of Source is T_icp_inverse
    try:
        T_est = np.linalg.inv(T_icp)
    except np.linalg.LinAlgError:
        print(f"Singular matrix at index {idx}")
        continue

    # E. Calculate Errors
    # 1. Translation Error (Vector difference)
    t_gt = T_gt[:3, 3]
    t_est = T_est[:3, 3]
    err_trans = get_translation_error(t_est, t_gt)
    
    # 2. Rotation Error (Angular difference)
    R_gt = T_gt[:3, :3]
    R_est = T_est[:3, :3]
    err_rot = get_rotation_error(R_est, R_gt)
    
    results.append({
        'id': row['id'],
        'filename': row['filename'],
        'icp_fitness': reg.fitness,
        'icp_rmse': reg.inlier_rmse,
        'error_translation_m': err_trans,
        'error_rotation_deg': err_rot,
        'matrix_icp': T_icp
    })
    
    if idx % 10 == 0:
        print(f"Sample {idx}: Rot Err={err_rot:.2f} deg, Trans Err={err_trans:.4f} m")

print("Analysis Complete.")

## 5. Statistics Report

In [ ]:
df_res = pd.DataFrame(results)

print("=== ACCURACY REPORT ===")
print(f"Mean Rotation Error:    {df_res['error_rotation_deg'].mean():.4f} degrees")
print(f"Max Rotation Error:     {df_res['error_rotation_deg'].max():.4f} degrees")
print("-" * 30)
print(f"Mean Translation Error: {df_res['error_translation_m'].mean():.6f} meters")
print(f"Max Translation Error:  {df_res['error_translation_m'].max():.6f} meters")
print("-" * 30)
print(f"Mean ICP Fitness:       {df_res['icp_fitness'].mean():.4f}")

## 6. Visualization
This function visualizes the **Alignment**.
* **Grey:** The Reference (Sample 0).
* **Red:** The Source Cloud (Transformed by ICP).

If the alignment is correct, the **Red** cloud should perfectly overlap the **Grey** cloud.

In [ ]:
def visualize_result(sample_id):
    row = df_res[df_res['id'] == sample_id]
    if row.empty:
        print("Sample not found.")
        return
    
    # Get Data
    filename = row.iloc[0]['filename']
    icp_matrix = row.iloc[0]['matrix_icp']
    
    # Load Source
    pcd_src = load_and_preprocess(os.path.join(SCANS_DIR, filename))
    
    # Apply ICP Transformation (This moves Source ON TOP of Target)
    pcd_src.transform(icp_matrix)
    
    # Colors
    pcd_src.paint_uniform_color([1, 0, 0])      # Red (Aligned Source)
    
    pcd_tgt_vis = o3d.geometry.PointCloud(pcd_target)
    pcd_tgt_vis.paint_uniform_color([0.5, 0.5, 0.5]) # Grey (Reference)
    
    print(f"Visualizing Sample {sample_id} - Press 'q' to close window.")
    
    o3d.visualization.draw_geometries(
        [pcd_tgt_vis, pcd_src],
        window_name=f"Result Sample {sample_id}",
        width=800, height=600,
        left=50, top=50
    )

# Visualize a specific sample (e.g., Sample 5)
visualize_result(5)